In [1]:
#---Data---#
#Agencies
agency = collect(1:10)

#Position of agencies, (x, y) coordinates
position = [0 0; 20 20; 18 10; 30 12; 35 0; 33 25; 5 27; 5 10; 11 0; 2 15]

#Car requirement of agencies 
carsNeeded = [10, 6, 8, 11, 9, 7, 15, 7, 9, 12]

#Cars present currently
carsInitially = [8, 13, 4, 8, 12, 2, 14, 11, 15, 7]

#Transportation cost per car per mile
mileCost = 0.50

#Ratio of road distance to Euclidean distance
ratio = 1.3


#---Model---#
using JuMP
model = Model()

#Flow of cars: [source, target, amount]
@defVar(model, flow[agency, agency] >= 0)

#Cost of transporting a car between two agencies
@defExpr(cost[s in agency, r in agency],  sqrt( (position[s, 1] - position[r, 1])^2 
+ (position[s, 2] - position[r, 2])^2)*ratio*mileCost*flow[s,r])

#Inflows to an agency
@defExpr(inflow[r in agency], sum{flow[s, r], s in agency})

#Outflows out of an agency
@defExpr(outflow[s in agency], sum{flow[s, r], r in agency})

#Cars present at an agency
@defExpr(carsNow[a in agency], carsInitially[a] + inflow[a] - outflow[a])

#Agencies must get the cars they need
@addConstraint(model, meetNeed[a in agency], carsNow[a] >= carsNeeded[a])

#Constraint on outflow of cars from an agency
@addConstraint(model, supply[a in agency], outflow[a] <= carsInitially[a])

#Minimize transportation cost
@setObjective(model, Min, sum(cost))

solve(model)

#Display non-zero flows
println("Flow of cars: ")
for s in agency
    for r in agency
        if(getValue(flow[s,r]) > 0)
            println("Agency $s to agency $r: ", getValue(flow[s,r]))
        end
    end
end

Flow of cars: 
Agency 2 to agency 3: 1.0
Agency 2 to agency 6: 5.0
Agency 2 to agency 7: 1.0
Agency 5 to agency 4: 3.0
Agency 8 to agency 10: 5.0
Agency 9 to agency 1: 2.0
Agency 9 to agency 3: 3.0
Agency 9 to agency 8: 1.0
